In [10]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [11]:
import pandas as pd
from lgbm_classifier import LGBMClassifierModel
from lightgbm import LGBMClassifier

In [12]:
train_df = pd.read_parquet("data/training_set_processed.parquet")
test_df = pd.read_parquet("data/test_set_processed.parquet")

In [13]:
lgbm_class_model = LGBMClassifierModel(
    df = train_df,              # Only initialized to get column names
    objective="binary",         # binary click prediction
    boosting_type="gbdt",       # gradient‑boosted decision trees
    n_estimators=1000,          # upper cap on number of trees
    learning_rate=0.05,
)
X_train, y_train, X_val, y_val, val_idx = lgbm_class_model.format_data(train_df)

In [14]:
lgbm_class_model.fit(
    X_train,
    y_train,
    X_val,
    y_val,
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 177546, number of negative: 3789131
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.287477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11308
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044759 -> initscore=-3.060662
[LightGBM] [Info] Start training from score -3.060662
Training until validation scores don't improve for 50 rounds
[20]	valid_0's auc: 0.709612	valid_0's binary_logloss: 0.171356
[40]	valid_0's auc: 0.718063	valid_0's binary_logloss: 0.168822
[60]	valid_0's a

In [16]:
lgbm_class_model.evaluate_validation(
    X_val,
    y_val,
    train_df,
    val_idx
)

Validation AUC: 0.7427294881560009


/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2/lgbm_classifier.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = predictions


Validation NDCG@5: 0.3759917362136209


(0.7427294881560009, 0.3759917362136209)

In [17]:
lgbm_class_model.get_ndcg_score(train_df)

Validation NDCG@5: 0.44144673982070093


0.44144673982070093

In [18]:
lgbm_class_model.get_final_predictions(test_df)

,srch_id,prop_id,score
9,1,54937,0.132118
5,1,28181,0.106891
12,1,61934,0.087581
23,1,99484,0.086846
1,1,5543,0.056554
...,...,...,...
4959177,332787,29018,0.202499
4959178,332787,32019,0.200721
4959182,332787,99509,0.188598
4959181,332787,94437,0.156132
